<a href="https://colab.research.google.com/github/charu11/ML-with-pytorch/blob/ml/CNN_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os
import cv2
import numpy as np
from tqdm import tqdm

In [39]:
REBUILD_DATA = True


class DogsVSCats():
    IMG_SIZE = 50
    CATS = '/content/drive/My Drive/cats_and_dogs_filtered/train/cats' 
    DOGS = '/content/drive/My Drive/cats_and_dogs_filtered/train/dogs'
    TESTING = '/content/drive/My Drive/cats_and_dogs_filtered/validation'
    LABELS = {CATS: 0, DOGS: 1}
    training_data = []

    catcount = 0
    dogcount = 0

    def make_training_data(self):
        for label in self.LABELS:
            print(label)
            for f in tqdm(os.listdir(label)):
                if "jpg" in f:
                    try:
                        path = os.path.join(label, f)
                        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
                        img = cv2.resize(img, (self.IMG_SIZE, self.IMG_SIZE))
                        self.training_data.append([np.array(img), np.eye(2)[self.LABELS[label]]])  # do something like print(np.eye(2)[1]), just makes one_hot 
                        #print(np.eye(2)[self.LABELS[label]])

                        if label == self.CATS:
                            self.catcount += 1
                        elif label == self.DOGS:
                            self.dogcount += 1

                    except Exception as e:
                        pass
                        #print(label, f, str(e))

        np.random.shuffle(self.training_data)
        np.save("training_data.npy", self.training_data)
        print('Cats:',dogsvcats.catcount)
        print('Dogs:',dogsvcats.dogcount)

if REBUILD_DATA:
    dogsvcats = DogsVSCats()
    dogsvcats.make_training_data()





  3%|▎         | 28/1010 [00:00<00:03, 279.54it/s]

/content/drive/My Drive/cats_and_dogs_filtered/train/cats


  3%|▎         | 27/1000 [00:00<00:03, 262.10it/s]

/content/drive/My Drive/cats_and_dogs_filtered/train/dogs


100%|██████████| 1000/1000 [00:03<00:00, 263.62it/s]

Cats: 1010
Dogs: 1000


In [0]:
training_data = np.load('training_data.npy', allow_pickle=True )
print(len(training_data))



In [0]:
import torch


X = torch.Tensor([i[0] for i in training_data]).view(-1, 50,50)
X = X/255
y = torch.Tensor([i[1] for i in training_data])


In [0]:
import matplotlib.pyplot as plt


plt.imshow(X[0],cmap='gray')


In [55]:
print(y[0])

tensor([0., 1.])


In [62]:
import torch
import torch.nn as nn
import torch.nn.functional as F



class Net(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(1, 32, 5)
    self.conv2 = nn.Conv2d(32, 64, 5)
    self.conv3 = nn.Conv2d(64, 128, 5)

    x = torch.randn(50,50).view(-1,1, 50,50)
    self._to_linear = None
    self.convs(x)


    self.fc1 = nn.Linear(self._to_linear, 512)  # flattening
    self.fc2 = nn.Linear(512, 2)  # 512 in, 2 out but we have 2 classes cats and dogs




  def convs(self, x):
    
    # max pooling over 2*2
    x = F.max_pool2d(F.relu(self.conv1(x)), (2,2))
    x = F.max_pool2d(F.relu(self.conv2(x)), (2,2))
    x = F.max_pool2d(F.relu(self.conv3(x)), (2,2))

    if self._to_linear is None:
      self._to_linear = x[0].shape[0]*x[0].shape[1]*x[0].shape[2]
    return x



  def forward(self, x):
    x = self.convs(x)
    x = x.view(-1, self._to_linear)
    x = F.relu(self.fc1(x))
    x = self.fc2(x)

    return F.softmax(x, dim=1)


net = Net()
print(net)



    
    







Net(
  (conv1): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=512, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=2, bias=True)
)


In [0]:
import torch.optim as optim

optimizer = optim.Adam(net.parameters(), lr=0.001)
loss_function = nn.MSELoss

X = torch.Tensor([i[0] for i in training_data]).view(-1, 50,50)
X = X/255.0
y = torch.Tensor([i[1] for i in training_data])


In [68]:

VAL_PCT = 0.1
val_size = int(len(X)*VAL_PCT)
print(val_size)

201


In [71]:
train_X = X[:-val_size]
train_y = y[:-val_size]

test_X = X[-val_size]
test_y = y[-val_size]

print(len(train_X), len(test_X))

1809 50


In [0]:
BATCH_SIZE = 100
EPOCHS = 8

for epoch in range(EPOCHS): 
  for i in tqdm(range(0, len(train_X), BATCH_SIZE)):
      batch_X = train_X[i:i+BATCH_SIZE].view(-1, 1, 50, 50)
      batch_y = train_y[i:i+BATCH_SIZE]

      net.zero_grad()

      outputs = net(batch_X)
      loss = loss_function(outputs, batch_y)
      loss.backward()
      optimizer.step()

  print(f'Epoch: {epoch}. Loss: {loss}')
  